# The Fashion MNIST - multiclass + CNN
Now we are back to the **fashion MNIST dataset**. This time we will use a CNN to process the images. For every image, we want to classify it into **10 distinct clothing categories**.

The big difference between processing images using a CNN compared to a dense network is that a **CNN** will process the images **"as images"**. This roughly means that we process the image by looking at pixels close together in the image and detect patterns in pixel values.

In this notebook, we will process the image by using a **convolutional layers**. The **more layers we add, the more complex patterns** can be detected. This approach is more reasonable than a naive approach (dense) since we use way **fewer parameters** by applying the same filter on many locations in the image.

Convolutional layers in Keras expect the input to have 3D dimensions `(width, height, colours)` but in the Fashion MNIST dataset each image is 2D, `(width, height)`. This is because the images are grayscale (range from black to white). If an image has colour, which consists of 3 colour values per pixel; Red, Green, Blue, we need to add an additional dimension to our tensor, making it 3D. This means that we need to add one dimension to our data. We will use the reshape function to achieve this. This means that our whole dataset will become a 4D tensor with dimensions `(examples, width, height, colours)` the training set this will have dimension `(60000, 28, 28, 1)`.

We will preprocess the data by first scaling the data using **Min/Max scaling**, map the labels to their **one-hot encoding representation** and then reshape the training data. Today we will not split the dataset further as we are provided with the train and test sets. After preprocessing the data we will then create a simple CNN to solve the multiclass classification task by using a **softmax layer** at the end which outputs a 10 class probability distribution and train the CNN using **categorical_crossentropy**. 

Let's load the data set:

In [ ]:
library(tensorflow)
library(keras)
source("06-helpers.R")

use_multi_cpu(2L)

data <- dataset_fashion_mnist()

Let us quickly inspect the dimensions of our dataset so that we see how many examples we have and the size of the images.

In [ ]:
dim(data$train$x)
length(data$train$y)
dim(data$test$x)
length(data$test$y)

These are 60000 training images, each one is 28 by 28 pixels. These are grayscale images, so the "colour" ranges from 0 (black) to 255 (white). Let's print out a single pixel in the first image.

In [ ]:
data$train$x[1,15,15]

## 1. Preprocessing
As a first step in our preprocessing we scale the images by making sure that the maximum value each feature (pixel) takes is at most 1 and the minimum is 0. To do this use Min/Max scaling.

$$
x' = \frac{x - \min(\boldsymbol{x})}{\max(\boldsymbol{x}) - \min(\boldsymbol{x})}
$$

Where $x$ is a single example and $x'$ is our new scaled value. $\min(\boldsymbol{x})$ is the smallest value in the training set and $\max(\boldsymbol{x})$ is the largest value.

In this case we know that the minimum pixel value is 0 and the maximum is 255. This means that just need to divide each pixel value by 255.

## 1.1 Scale the train/test sets
Scale the train and test sets by dividing all of the pixel values by 255. For `data$train$x` store the result in `x_train_scaled` and for `data$test$x` store the result in `x_test_scaled`.

In [ ]:
<FILL IN>

## 1.2 Reshaping
Next we reshape our data. We need to add one extra dimension to our data, otherwise the `layer_conv_2d` in Keras will complain. For each image a `(28, 28)` we need to reshape it into `(28, 28, 1)`. This is a change "on paper" and will not change anything.

[`Reshape`](https://rstudio.github.io/reticulate/reference/array_reshape.html) `x_train_scaled` and `x_test_scaled` to `c(60000, 28, 28, 1)` and `c(10000, 28, 28, 1)`, respectively. Check your work using the dim function. Store the result as `x_train` and `x_test`.

In [ ]:
<FILL IN>

## 1.3 One hot-encoding
We also need to preprocess the labels of the images. Each image in your dataset is associated with a certain label, what type of clothing it is. 

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

These labels range from 0 to 9. We need to map these labels to their corresponding one-hot encodings. We do this using the [`to_categorical()`](https://www.rdocumentation.org/packages/keras/versions/2.2.4/topics/to_categorical) function.

Apply this function to `data$train$y` and `data$test$y` and store the output in `y_train` and `y_test`, respectively. Verify your work with the `dim` function.

In [ ]:
<FILL IN>

## 2. The models
Now we want to create a few simple CNN models. Let us start with a model which consists of a single convolutional layer.

## 2.1 A single conv layer
Create an initial model with the following properties.
- a 2d convolutional layer, `layer_conv_2d`, with `filters = 32`, `kernel_size = c(3, 3)`, `strides = c(2, 2)` and `padding = 'valid'`. For more information about the 2d conv layer see the [`layer_conv_2d`](https://www.rdocumentation.org/packages/keras/versions/2.2.4/topics/layer_conv_2d) documentation.
- After the `layer_conv_2d` apply the `layer_activation` with the `relu` activation function.
- Then flatten the output using `layer_flatten`.
- Then apply a `layer_dense` with 10 units and `softmax` activation.

In [ ]:
model <- keras_model_sequential() %>%
    <FILL IN>

cat(summary(model))

model %>% compile(
    optimizer = optimizer_adam(lr = 0.01),
    loss = "categorical_crossentropy",
    metric = "acc"
)

history <- model %>% fit(
    x = x_train,
    y = y_train,
    validation_split = 0.2,
    epochs = 15,
    batch_size = 256,
    callbacks=list(Progress$new())
)

plot(history)

Let us decipher this plot. 
- training loss continues to go down, this seems to imply that our model is sufficiently expressive.
- validation loss goes up in the end, we are overfitting.
- validation accuracy is fairly stable, our overfitting has not impacted our accuracy, yet.

Let us try to reduce overfitting in the next step.

## 2.2 Regularise
Let us add some dropout to our model in hopes of reducing overfitting. Copy the model from above and add `layer_dropout` after the `relu` activation. We used a dropout of `0.4` and discuss those results below.

Looking at this plot. We can see:
- Training loss seems to reduce slower now
- Validation loss is stable, thus less overfitting.

It seems that we could train this model longer and achieve better performance, but instead let us increase the complexity of the model by adding an additional convolutional layer. This will allow the model to detect more complex patterns which will undoubtably improve the performance of the model.

## 2.3 Two convolutional layers
Let us try increasing the model complexity by adding one more `layer_conv_2d` and `layer_activation` with `relu` to our model.
Copy the model from above and:
- add another `layer_conv_2d` and `layer_activation` with `relu` directly after the first `dropout` activation. Use the same parameters as first layer. Remove the `input_shape` parameter.
- Add dropout with 0.2 after the new `relu` activation.
- Reduce the dropout we specified earlier to 0.2.


From this plot we can see.
- Training loss and validation loss are close and decreasing slowly, this implies that we might want to increase model complexity to see if we can further improve our model.

We have improved our model somewhat from the initial model and we have without a doubt a better model than the dense network we trained in notebook 03a. You can confirm this be evaluating the model below.

We can continue improving this model, and we leave this as a bonus exercise.

In [ ]:
model %>% evaluate(x_test, y_test)

In this notebook we performed multi-class classification on the Fashion MNIST dataset. Today we processed the images using convolutional layers instead of a dense network. This allowed us to extra local pixel patterns and combine these patterns to create more complex patters. We started with a simple model using a single convolutional layer which we then slowly improved.

## Bonus exercise
Continue improving the model from 2.3. Try getting above 90% accuracy. Consider these options
- Add more convolutional layers (with activations)
- Add batch normalisation
- Increase the number of filters in later convolutional layers. 
- Add a dense later after all the convolutions but before the softmax.
- Use "same" padding in early convolutions.
- Adding more epochs